# Action Recognition @ UCF101  
**Due date: 11:59 pm on Nov. 19, 2019 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the amount of data. Consequently, a training sample is an image tuple that forms a 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training an end-to-end neural network from scratch whose computation is prohibitively expensive, we divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use any of the [pre-trained models](https://pytorch.org/docs/stable/torchvision/models.html) to extract features from each frame. Specifically, we recommend not to use the activations of the last layer as the features tend to be task specific towards the end of the network. 
    **hints**: 
    - A good starting point would be to use a pre-trained VGG16 network, we suggest first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    - Normalize your images using `torchvision.transforms` 
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    The mean and std. mentioned above is specific to Imagenet data
    
    ```
    More details of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a **dx25** sample as input (where **d** is the dimension of the extracted feature for each frame), and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy **(5 points)**. The baseline test accuracy for this data is 75%, and **10 points** out of 20 is for achieving test accuracy greater than the baseline. Moreover, you need to compare **(5 points)** the result of your network with that of support vector machine (SVM) (stacking the **dx25** feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas your pre-trained model might take **nxn** images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five **nxn** images, one at the image center and four at the corners and compute the **d**-dim features for each of them, and average these five **d**-dim feature to get a final feature representation for the raw image.
For example, VGG takes 224x224 images as inputs, so we take the five 224x224 croppings of the image, compute 4096-dim VGG features for each of them, and then take the mean of these five 4096-dim vectors to be the representation of the image.

In order to save you computational time, you need to do the classification task only for **the first 25** classes of the whole dataset. The same applies to those who have access to GPUs. **Bonus 10 points for running and reporting on the entire 101 classes.**


## Dataset
Download **dataset** at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar)(Image data for each video) and the **annos folder** which has the video labels and the label to class name mapping is included in the assignment folder uploaded. 


UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images/`  
  + each folder represents a video
  + the video/folder name to class mapping can be found using `annots/videos_labels_subsets.txt`, for e.g. `v_000001` belongs to class 1 i.e. `ApplyEyeMakeup`
  + each video folder contains 25 frames  



## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
cd 'gdrive/My Drive/CV/hw5/'

/content/gdrive/My Drive/CV/hw5


In [0]:
# import packages here
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random 
import time
from scipy import ndimage
import gc 
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from skimage.transform import rotate
import  tarfile


import torch.optim as optim
import random
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [0]:
# untarring the tar file to extract the vido folders
tar = tarfile.open("UCF101_images.tar", "r:")
tar.extractall()
tar.close()

---
---
## **Problem 1.** Feature extraction

In [0]:
foldernames = [name[9:] for name in glob.glob('./images/*')]
foldernames.sort()
raw_folders = dict(zip(range(1,len(foldernames)+1), foldernames))

f= open('./annos/videos_labels_subsets.txt')
video_labels_subset =([line.split() for line in f]) 

res = {video_labels_subset[i][0]: video_labels_subset[i][1] for i in range(len(video_labels_subset))} 
data_set_dict = {video_labels_subset[i][0]: video_labels_subset[i][2] for i in range(len(video_labels_subset))} 

In [7]:
import torch.optim as optim

vgg = torchvision.models.vgg16(pretrained = True)
vgg.classifier = nn.Sequential(*list(vgg.classifier.children())[:-3])     # Removing last three layers
vgg.cuda()            #Using GPU


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:42<00:00, 13.0MB/s]


In [0]:
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:

def load_dataset(No_of_classes, dataset_type = '1'):
  
  path = './images/'
  data = []
  label = []

  for id, class_name in raw_folders.items():
    #print(id,class_name)
    if int(res[class_name]) < No_of_classes + 1:
          img_path_class = glob.glob(path + class_name + '/*.jpg')
          if data_set_dict[class_name] == dataset_type:
            #size = size + 1
            #if video_labels_subset[id][0] == class_name:
            label.append(res[class_name])  
            for filename in img_path_class:
                im = Image.open(filename)
                #data.append(im)

                im1 = Image.Image.crop(im,box = (0,0,224,224))
                im2 = Image.Image.crop(im,box = (116,0,340,224))
                im3 = Image.Image.crop(im,box = (0,32,224,256))
                im4 = Image.Image.crop(im,box = (116,32,340,256))
                im5 = Image.Image.crop(im,box = (58,16,282,240))

                normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                prep = transforms.Compose([ transforms.ToTensor(), normalize ])

                im1 = prep(im1)
                im2 = prep(im2)
                im3 = prep(im3)
                im4 = prep(im4)
                im5 = prep(im5)


                im1 = (im1).unsqueeze(0)
                im2 = (im2).unsqueeze(0)
                im3 = (im3).unsqueeze(0)
                im4 = (im4).unsqueeze(0)
                im5 = (im5).unsqueeze(0)

                
                im1 = (im1).cuda()
                im2 = (im2).cuda()
                im3 = (im3).cuda()
                im4 = (im4).cuda()
                im5 = (im5).cuda()

                with torch.no_grad():
                  im1 = vgg(im1)
                  im2 = vgg(im2)
                  im3 = vgg(im3)
                  im4 = vgg(im4)
                  im5 = vgg(im5)

                im = (im1 + im2 + im3 + im4 + im5)/5

                data.append(im)
                gc.collect()
  return data,label

  
 



In [0]:
#Extracting features for Train Videos
data,label = load_dataset(No_of_classes, dataset_type = '1')

In [0]:
#Extracting features for Test Videos
test_data, test_label = load_dataset(No_of_classes=25, dataset_type = '2')

In [0]:
#Saving Extracted features in Pickle files.
import pickle
import pandas as pd 

with open("train_data.pkl", 'wb') as fiModel:
    pickle.dump(data, fiModel,protocol=pickle.HIGHEST_PROTOCOL)

with open("train_label.pkl", 'wb') as fiModel2:
    pickle.dump(label, fiModel2,protocol=pickle.HIGHEST_PROTOCOL)

with open("test_data.pkl", 'wb') as fiModel3:
    pickle.dump(test_data, fiModel3,protocol=pickle.HIGHEST_PROTOCOL)

with open("test_label.pkl", 'wb') as fiModel4:
    pickle.dump(test_label, fiModel4,protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# Extracting features from Pickle files
import pickle
file = open('train_data.pkl', 'rb')
train_data = pickle.load(file)
file.close()

file = open('train_label.pkl', 'rb')
train_label = pickle.load(file)
file.close()

file = open('test_data.pkl', 'rb')
test_data = pickle.load(file)
file.close()

file = open('test_label.pkl', 'rb')
test_label = pickle.load(file)
file.close()

Reshaping the Train and Test data

In [0]:
train = np.array(train_data)
train = train.reshape(2409,25)

tt =[]
for im in train:
  for im2 in im:
    for im3 in im2:
      im3 = im3.cpu()
      tt.append(im3.numpy())

train = np.array(tt)
train_d = train.reshape(2409,25,4096)    

In [0]:
test = np.array(test_data)
test = test.reshape(951,25)

tt =[]
for im in test:
  for im2 in im:
    for im3 in im2:
      im3 = im3.cpu()
      tt.append(im3.numpy())

test = np.array(tt)
test_d = test.reshape(951,25,4096)  

***
***
## **Problem 2.** Modelling

* ##### **Print the size of your training and test data**

In [219]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :',train_d.shape )
print('Shape of test/validation data is :',test_d.shape )

Shape of training data is : (2409, 25, 4096)
Shape of test/validation data is : (951, 25, 4096)


In [0]:
# \*write your codes for modelling using the extracted feature (You can use multiple cells, this is just a place holder)

torch.manual_seed(1)

class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1, num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda(),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).cuda())

    def forward(self, input):
        lstm_out, self.hidden = self.lstm(input.view(output_dim,self.batch_size,self.input_dim))
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(1,-1)

batch_size = 1
num_of_layers = 1
input_size = 4096         # input size
hidden_layers = 2000      # Number of hidden layers
output_dim = 25           # Number of Classes i.e 25

model = LSTM(input_size, hidden_layers, batch_size=batch_size, output_dim=output_dim, num_layers=num_of_layers)


In [0]:
# Random index generation for shuffling the data
ind = [i for i in range(2409)]
index_generation = random.sample(ind, 2408)

In [0]:
train_d_t = torch.FloatTensor(train_d)
test_d_t = torch.FloatTensor(test_d)

labels = [int(i) for i in train_label] 
testlbs = [int(i) for i in test_label] 

In [39]:
model.cuda()

learning_rate = 0.01
loss_fn = nn.CrossEntropyLoss() 
optimiser = optim.SGD(model.parameters(), lr=learning_rate) 

num_of_epochs = 10
start_time = time.time()
for epoch in range(num_of_epochs):

    loss_epoch = 0
    for i in index_generation:            # With Shuffling
    #for i in range(0,len(train_d_t)):    # Without Shuffling

        action_label = []
        vid = train_d_t[i]                #Taking one video folder at a time
        vid = vid.cuda()                  #alloting GPU
        
        action_label.append(int(labels[i])-1)   #subtracting 1 becuase our model predict between 0-24 for 25 classes.
        action_label = torch.LongTensor(action_label)
        action_label =action_label.cuda()  

        model.zero_grad()                     # zero the parameter gradients
        model.hidden = model.init_hidden()    # Initialise hidden state
        y_pred = model(vid)                   # Training the model
        loss = loss_fn(y_pred, action_label)  # Calculating Loss 

        loss_epoch += loss.item()

        optimiser.zero_grad()  # Zero out gradient
        loss.backward()        # Backward pass
        optimiser.step()   

    print('Epoch: ',epoch , 'Loss incurred: ',loss_epoch/len(train_d_t))
time1 = time.time() - start_time
print('Time Consumed to train the model: ',time1)

Epoch:  0 Loss incurred:  1.0021480861270313
Epoch:  1 Loss incurred:  0.3394117249448847
Epoch:  2 Loss incurred:  0.1384851119982449
Epoch:  3 Loss incurred:  0.04763858632063559
Epoch:  4 Loss incurred:  0.01846290594712982
Epoch:  5 Loss incurred:  0.010244914039431296
Epoch:  6 Loss incurred:  0.003850103958655611
Epoch:  7 Loss incurred:  0.002614160650941335
Epoch:  8 Loss incurred:  0.0020210219399470816
Epoch:  9 Loss incurred:  0.0016474464514682084
Time Consumed to train the model:  332.0369248390198


---
---
## **Problem 3.** Evaluation

Predicting actions for Test and Train Data

In [40]:
# \*write your codes for evaluation (You can use multiple cells, this is just a place holder)
start_time = time.time()

with torch.no_grad():
    correct = 0
    for i in range(0,len(test_d_t)):
        action_label=[]
        vid = test_d_t[i].cuda()
        action_label.append(int(testlbs[i])-1)
        action_label = torch.LongTensor(action_label).cuda()
        
        model.hidden = model.init_hidden()
        pred = model(vid)
        _, predicted_label = torch.max(pred.data, 1)
        correct += (predicted_label == action_label).sum().item()
    
    accuracy_test = correct/len(test_d_t)
time2 = time.time() - start_time
print('Time Consumed to predict test data: ',time2)

Time Consumed to predict test data:  4.683665990829468


In [41]:
start_time = time.time()

with torch.no_grad():
    correct = 0
    for i in range(0,len(train_d_t)):
        action_label=[]
        vid = train_d_t[i].cuda()
        action_label.append(int(labels[i])-1)
        action_label = torch.LongTensor(action_label).cuda()
        
        model.hidden = model.init_hidden()
        pred = model(vid)
        _, predicted_label = torch.max(pred.data, 1)
        correct += (predicted_label == action_label).sum().item()
    
    accuracy_train = correct/len(train_d_t)
time3 = time.time() - start_time
print('Time Consumed to predict train data: ',time3)

Time Consumed to predict train data:  11.848022222518921


* ##### **Print the train and test accuracy of your model** 

In [42]:
# Don't hardcode the train and test accuracy
print('Training accuracy is %2.3f %% :' %(accuracy_train*100) )
print('Test accuracy is %2.3f %%:' %(accuracy_test*100) )

Training accuracy is 100.000 % :
Test accuracy is 81.914 %:


* ##### **Print the train and test and test accuracy of SVM** 

Reshaping the Data: feature of 25 images into one vector

In [0]:
train_svm = torch.FloatTensor(train).view(2409, 102400)
test_svm = torch.FloatTensor(test).view(951, 102400)
tr_labels_svm = [int(i)-1 for i in train_label] 
testlbs_svm = [int(i)-1 for i in test_label] 


In [11]:
from sklearn.svm import SVC
start_time = time.time()
svm_model = SVC()
svm_model.fit(train_svm, tr_labels_svm)
time4 = time.time() - start_time
print('Time Consumed to train the SVM Model: ',time4)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Time Consumed to train the SVM Model:  1193.7089142799377


In [48]:
start_time = time.time()
pred = svm_model.predict(test_svm)
time5 = time.time() - start_time
print('Time Consumed to predict test data using SVM Model: ',time5)

Time Consumed to predict test data using SVM Model:  400.33932065963745


In [16]:
start_time = time.time()
train_pred = svm_model.predict(train_svm)
time6 = time.time() - start_time
print('Time Consumed to predict train data using SVM Model: ',time6)

Time Consumed to predict train data using SVM Model:  1044.6530919075012


In [49]:
# Don't hardcode the train and test accuracy
print('Training accuracy is %2.3f %% :' %(accuracy_score(train_pred, tr_labels_svm)*100) )
print('Test accuracy is %2.3f %% :' %(accuracy_score(pred, testlbs_svm)*100) )

Training accuracy is 99.958 % :
Test accuracy is 79.811 % :


## **Problem 4.** Report

Time Consumptions

In [51]:
print('\nFor LSTM:\n')
print('Time Consumed to train the model: ',time1)
print('Time Consumed to predict test data: ',time2)
print('Time Consumed to predict train data: ',time3)
print('\nFor SVM:\n')
print('Time Consumed to train the SVM Model: ',time4)
print('Time Consumed to predict test data using SVM Model: ',time5)
print('Time Consumed to predict train data using SVM Model: ',time6)


For LSTM:

Time Consumed to train the model:  332.0369248390198
Time Consumed to predict test data:  4.683665990829468
Time Consumed to predict train data:  11.848022222518921

For SVM:

Time Consumed to train the SVM Model:  1193.7089142799377
Time Consumed to predict test data using SVM Model:  400.33932065963745
Time Consumed to predict train data using SVM Model:  1044.6530919075012


Accuracies

In [50]:
print('\nFor LSTM:\n')
print('Training accuracy is %2.3f %% :' %(accuracy_train*100) )
print('Test accuracy is %2.3f %%:' %(accuracy_test*100) )
print('\nFor SVM:\n')
print('Training accuracy is %2.3f %% :' %(accuracy_score(train_pred, tr_labels_svm)*100) )
print('Test accuracy is %2.3f %% :' %(accuracy_score(pred, testlbs_svm)*100) )


For LSTM:

Training accuracy is 100.000 % :
Test accuracy is 81.914 %:

For SVM:

Training accuracy is 99.958 % :
Test accuracy is 79.811 % :


## **Bonus**


* ##### **Print the size of your training and test data**

In [0]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

* ##### **Modelling and evaluation**

In [0]:
#Write your code for modelling and evaluation

## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.
The naming convention for report is **Surname_Givenname_SBUID_report*.pdf**

When submitting your .zip file through blackboard, please
-- name your .zip file as **Surname_Givenname_SBUID_hw*.zip**.

This zip file should include:
```
Surname_Givenname_SBUID_hw*
        |---Surname_Givenname_SBUID_hw*.ipynb
        |---Surname_Givenname_SBUID_hw*.pdf
        |---Surname_Givenname_SBUID_report*.pdf
```

For instance, student Michael Jordan should submit a zip file named "Jordan_Michael_111134567_hw5.zip" for homework5 in this structure:
```
Jordan_Michael_111134567_hw5
        |---Jordan_Michael_111134567_hw5.ipynb
        |---Jordan_Michael_111134567_hw5.pdf
        |---Jordan_Michael_111134567_report*.pdf
```

The **Surname_Givenname_SBUID_hw*.pdf** should include a **google shared link**. To generate the **google shared link**, first create a folder named **Surname_Givenname_SBUID_hw*** in your Google Drive with your Stony Brook account. 

Then right click this folder, click ***Get shareable link***, in the People textfield, enter two TA's emails: ***bo.cao.1@stonybrook.edu*** and ***sayontan.ghosh@stonybrook.edu***. Make sure that TAs who have the link **can edit**, ***not just*** **can view**, and also **uncheck** the **Notify people** box.

Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in blackboard is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in blackboard. 

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

Be aware that your code will undergo plagiarism check both vertically and horizontally. Please do your own work.